Dataset() 클래스

    다양한 종류의 데이터셋 지원에 필요한 공통 기능을 제공하면서 여러가지 데이터셋 클래스들의 기반 클래스 역할

In [1]:
import os

os.chdir('C:\\Users\\gowra\\1_멋쟁이 사자처럼\\2. 신경망 기법\\신경망(클래스_프로그램_제작_프로젝트)/code/')

In [2]:
%run mathutil.ipynb

In [ ]:
class Dataset(object):
    def __init__(self, name, mode):  # 초기화 (이름/유형)
        self.name = name
        self.mode = mode

    def __str__(self):  # 문자열 초기화 메서드
        return '{}({}, {}+{}+{})'.format(self.name, self.mode, len(self.tr_xs),
                                         len(self.te_xs), len(self.va_xs)) 

    @property  # 함수가 아닌 속성으로 취급해주는 데코레이터
    def train_count(self):
        return len(self.tr_xs)

    메서드는 초기화  __init__() 메서드, 문자열화  __str__() 메서드가 정의

    초기화 메서드에서는 데이터셋 이름과 출력의 유형을 의미하는 모드값을 챙겨 저장합니다. 
    모드(mode)의 경우 회귀 분석, 이진 분류, 다중 선택 분류를 나타내는 'regression', 'binary', 'select' 로 구분하여 문제를 해결

    문자열화 메서드의 경우 출력 유형을 설정할 수 있는데, 
    데이터셋, 모드유형, 학습 데이터의 크기, 테스트 데이터의 크기, 검증 데이터의 크기 이렇게 다섯가지 정보를 각각 표현

    학습 데이터 셋의 수를 반환하는 train_count() 메서드 
    실제로는 함수 메서드이지만 함수가 아닌 속성으로 취급되며 x.train_count() 형식 대신 x.train_count 형식으로 접근

    MlpModel() 클래스의 train() 메서드에서 batch_count = int(self.dataset.train_count / batch_size) 의 구조를 살펴보면
    self.dataset.train_count 처럼 함수의 형식이 아니지만 접근이 가능한 것을 확인할 수 있습니다.


#### 학습 데이터를 분할 dataset_get_train_data

In [ ]:
def dataset_get_train_data(self, batch_size, nth):
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size

    tr_X = self.tr_xs[self.indices[from_idx:to_idx]]
    tr_Y = self.tr_ys[self.indices[from_idx:to_idx]]

    return tr_X, tr_Y


def dataset_shuffle_train_data(self, size):
    self.indices = np.arange(size)
    np.random.shuffle(self.indices)

Dataset.get_train_data = dataset_get_train_data
Dataset.shuffle_train_data = dataset_shuffle_train_data

    학습 데이터를 분할하는 과정은 위에서 살펴본 전체 학습 데이터의 수인 train_count를 우리가 설정한 batch_size 로 나누게 되면 학습 데이터가 batch_count로 얼마나 나뉘어 지는지 알 수 있다. 

    미니배치 단위를 인덱스로 구분하고, 구분되어진 인덱스 범위에서 tr_X,  tr_Y 를 구분하여 줍니다. 

    self.indices 는 아래 메서드를 통해 한 차례 무작위로 섞인 인덱스로, 그 인덱스의 범위를 슬라이싱하여 저장

    반복문  을 활용하여 batch_count를 순차적으로 nth 에 할당, 
    그리고 사전에 정의된 batch_size 값과의 연산을 통해 batch_size로 지정한 크기만큼 from_idx와 to_idx를 반복적으로 구해줄 수 있게된다.

    매번 같은 패턴으로 에폭값이 늘어나게 되면 학습은 제대로 진행되지 않습니다.
    그래서 한 차례 에폭이 진행 될 때마다 데이터를 무작위로 섞어주는 과정이 필요하게 됩니다

    dataset_shuffle_train_data() 메서드에서 정의

    batch_size와 batch_count의 곱한 값을 numpy의 arange()를 통해 실제 사용되는 학습 데이터를 인덱스로 생성하고, 
    이렇게 생성된 인덱스를 np.random.shuffle()를 통해 무작위로 섞어 제공


#### 테스트 데이터와 검증 데이터에 대한 분할과정 dataset_get_test_data

In [ ]:
def dataset_get_test_data(self):
    return self.te_xs, self.te_ys

Dataset.get_test_data = dataset_get_test_data

    dataset_get_test_data() 의 경우 mlp_model_test() 메서드에서 사용되어지는 메서드로, 
    별 다른 처리 없이 테스트 데이  터인 te_xs, te_ys 를 전달 받게 됩니다. 

    마지막 코드를 보면 하나의 메서드를 Dataset() 클래스의 두 메서드로 정의되는 것을 확인할 수 있다. 

    이 방식은  메서드를 외부에서 정의하고 클래스의 내부에 같은 기능을 하는 메서드를 다른 이름으로 정의할 수 있게 해줍니다.

#### 검증데이터 분할 / 시각화 dataset_get_validate_data

In [ ]:
def dataset_get_validate_data(self, count):
    self.va_indices = np.arange(len(self.va_xs))
    np.random.shuffle(self.va_indices)

    va_X = self.va_xs[self.va_indices[0:count]]
    va_Y = self.va_ys[self.va_indices[0:count]]

    return va_X, va_Y

Dataset.get_validate_data = dataset_get_validate_data
Dataset.get_visualize_data = dataset_get_validate_data

    검증 데이터 분할 메서드 get_validate_data()
    mlp_model_train() 메서드에서 조건에 만족한 경우에만 아래와 같은 방식으로 수행

    원하는 수 만큼의 count 매개 변수를 전달받게 되는데, 
    우선 self.va_xs는 이후에 살펴보겠지만 전체 데이터에서 원하는 비율만큼의 va_ratio를 설정하여 얻어진 검증 데이터의 독립 변수 데이터라 할 수 있습니다.
    그래서 np.arange()를 통해 검증 데이터의 인덱스를 확보하고, 그 인덱스를 한번 무작위로 섞어 줍니다.

    그리고 검증 데이터셋인 self.va_xs에 무작위로 섞인 인덱스 self.va_indices 로 접근하여 앞서 설정한 count 수 만큼의 데이터를 va_X에 할당하여 줍니다. 
    결과적으로  va_ratio비율의 검증 데이터셋에서 무작위로 섞인 count 수 만큼의 검증 데이터를 va_X에 할당한 것 입니다. 

    종속 변수도 같은 방식으로 하여 va_Y 를 돌려받아 줄 수 있습니다. 



#### 변수들을 정의 dataset_shuffle_data

In [ ]:
def dataset_shuffle_data(self, xs, ys, tr_ratio=0.8, va_ratio=0.05):
    data_count = len(xs)

    tr_cnt = int(data_count * tr_ratio / 10) * 10
    va_cnt = int(data_count * va_ratio)
    te_cnt = data_count - (tr_cnt + va_cnt)

    tr_from, tr_to = 0, tr_cnt
    va_from, va_to = tr_cnt, tr_cnt + va_cnt
    te_from, te_to = tr_cnt + va_cnt, data_count

    indices = np.arange(data_count)
    np.random.shuffle(indices)

    self.tr_xs = xs[indices[tr_from:tr_to]]
    self.tr_ys = ys[indices[tr_from:tr_to]]
    self.va_xs = xs[indices[va_from:va_to]]
    self.va_ys = ys[indices[va_from:va_to]]
    self.te_xs = xs[indices[te_from:te_to]]
    self.te_ys = ys[indices[te_from:te_to]]

    self.input_shape = xs[0].shape
    self.output_shape = ys[0].shape
    
    return indices[tr_from:tr_to], indices[va_from:va_to], indices[te_from:te_to]

Dataset.shuffle_data = dataset_shuffle_data

    tr_xs, te_xs, va_xs … 등의 변수들을 정의
    Dataset_chap123.ipynb 파일의 각 데이터셋을 다루는 클래스들에서 사용되어 집니다. 

    전체 데이터의 독립 변수와 종속변수, 학습 데이터 비율, 검증 데이터 비율값을 전달받아 
    학습 데이터, 검증 데이터, 테스트 데이터로 각각 나눠지게 된다.

    전체 데이터의 독립 변수인 xs에 len()를 적용하여 data_count 값을 구해줍니다

    구한 값과 tr_ratio,  va_ratio 를 활용하여 학습 데이터(tr_cnt)와 검증 데이터(va_cnt)의 수를 구해주고, 
    전체 수인 data_count 에서 위의 값을 빼주게 되면 자연스럽게 테스트 데이터의 수(te_cnt)를 얻어낼 수 있게 됩니다. 

    인덱스로  나누기위해 각 데이터 별로 시작 지점과 끝 지점을 지정하는 과정을 수행해줍니다. 
    그리고 데이터를 할당하기 위하여 전체 데이터의 인덱스를 np.arange()로 구해주고, 
    그 값을 np.random.shuffle()로 무작위로 섞어주겠습니다.
    
    학습 데이터, 검증 데이터, 테스트 데이터를 독립 변수와 종속 변수로 구분하여 저장
    MlpModel() 클래스의 파라미터 초기화 메서드를 정의하는 과정에서 input_shape과 output_shape를 Dataset() 클래스에서 정의한다 하였습니다.
    이 메서드에서는 독립 변수와 종속 변수를 받아왔기에 각 xs, ys의 첫 번째 행의 열 개수 즉, 입출력 벡터의 크기를 설정해 줄 수가 있습니다.  
    마지막으로 반환값은 각 데이터들의 원래 위치를 알 필요가 있을 때 를 대비하여 준비하였습니다. 


#### 순전파 과정⭐

In [ ]:
def dataset_forward_postproc(self, output, y, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        diff = output - y
        square = np.square(diff)
        loss = np.mean(square)
        aux = diff
    elif mode == 'binary':
        entropy = sigmoid_cross_entropy_with_logits(y, output)
        loss = np.mean(entropy)
        aux = [y, output]
    elif mode == 'select':
        entropy = softmax_cross_entropy_with_logits(y, output)
        loss = np.mean(entropy)
        aux = [output, y, entropy]
        
    return loss, aux

Dataset.forward_postproc = dataset_forward_postproc

    MlpModel() 클래스에서 정의되는 mlp_forward_postproc() 메서드에서는 ‘입력값’으로 신경망의 예측인 output 과 실제 정답인 y 가 들어오게 되었을 때, 바로 손실을 구하지 않고 문제의 성격에 맞게 동작을 수행

    dataset_forward_postproc() 이 메서드의 경우 세 가지 문제에 대한 손실과 이후 손실에 대한 역전파를 수행하기 위한 보조 지표를 반환시켜 주었습니다. 
    수행 방식은 init 초기화 메서드에서 정의된 mode 를 전달받아 if, else 조건문을 활용하여 mode 의 성격에 맞는 손실을 구하는 과정으로 정의합니다. 
    그리고 손실함수의 역전파 과정을 위한 aux 도 챙겨주도록 하였습니다.

    dataset_chap123.ipynb 에서 살펴볼 각 데이터별 클래스에서 Dataset() 클래스를 상속받아 데이터 이름과 모드를 문제 성격에 맞게 설정 후 
    클래스 변수화 시켜 줄 수 있고, 이를 MlpModel() 클래스의 dataset 에 할당시켜 다층 퍼셉트론 신경망 구축을 위한 파라미터 초기화 과정을 수행시켜 줍니다. 
    그리고 이는 Model() 클래스에 상속되어 있기에 Model() 클래스의 exec_all() 메서드를 적용하여 학습, 테스트, 시각화 과정을 순차적으로 수행


#### 손실 함수의 기울기를 구하는 과정

In [ ]:
def dataset_backprop_postproc(self, G_loss, aux, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        diff = aux
        shape = diff.shape

        g_loss_square = np.ones(shape) / np.prod(shape)
        g_square_diff = 2 * diff
        g_diff_output = 1

        G_square = g_loss_square * G_loss
        G_diff = g_square_diff * G_square
        G_output = g_diff_output * G_diff
    elif mode == 'binary':
        y, output = aux
        shape = output.shape

        g_loss_entropy = np.ones(shape) / np.prod(shape)
        g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    elif mode == 'select':
        output, y, entropy = aux

        g_loss_entropy = 1.0 / np.prod(entropy.shape)
        g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    
    return G_output

Dataset.backprop_postproc = dataset_backprop_postproc

    MlpModel() 클래스에서 mlp_backprop_postproc() 메서드로 정의되어 지지만, 메서드 정의 과정을 살펴보면 
    다시 dataset_backprop_postproc() 에서 G_output을 산출

    주어진 mode에 맞춰 회귀 분석, 이진 분류, 선택 분류에 대한 손실 함수의 역전파 과정을 정의

#### 평가와 예측

In [ ]:
def mlp_eval_accuracy(self, x, y, output=None):  
    if output is None:
        output, _ = self.forward_neuralnet(x)
    accuracy = self.dataset.eval_accuracy(x, y, output)
    return accuracy


In [ ]:
def dataset_eval_accuracy(self, x, y, output, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        mse = np.mean(np.square(output - y))
        accuracy = 1 - np.sqrt(mse) / np.mean(y)
    elif mode == 'binary':
        estimate = np.greater(output, 0)
        answer = np.equal(y, 1.0)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
    elif mode == 'select':
        estimate = np.argmax(output, axis=1)
        answer = np.argmax(y, axis=1)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
        
    return accuracy

Dataset.eval_accuracy = dataset_eval_accuracy

#### 예측 과정

In [ ]:
def mlp_get_estimate(self, x):
    output, _ = self.forward_neuralnet(x)  # 신경망의 결과
    estimate = self.dataset.get_estimate(output)  # 신경망의 결과를 가지고 예측
    
    return estimate


In [ ]:
def dataset_get_estimate(self, output, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        estimate = output
    elif mode == 'binary':
        estimate = sigmoid(output)
    elif mode == 'select':
        estimate = softmax(output)
        
    return estimate

Dataset.get_estimate = dataset_get_estimate

In [ ]:
def dataset_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    print('    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)'. \
          format(epoch, np.mean(costs), np.mean(accs), acc, time1, time2))

def dataset_test_prt_result(self, name, acc, time):
    print('Model {} test report: accuracy = {:5.3f}, ({} secs)\n'. \
          format(name, acc, time))

Dataset.train_prt_result = dataset_train_prt_result
Dataset.test_prt_result = dataset_test_prt_result

    dataset_train_prt_result()의 경우 mlp_model_train() 에서 조건문에 따른 report로 설정한 주기에 맞춰 수행 결과를 출력하게끔 설계된 메서드로,
    출력에 필요한 매개 변수를 전달받아 에폭 횟수, 학습 단계의 비용과 정확도, 검증 데이터의 정확도, 한 에폭마다 걸린시간과 누적 소요시간을 출력하게 됩니다. 
    단, 학습 데이터를 통한 비용과 정확도는 미니배치 마다 구해진 다수의 결괏값의 평균치로 출력하였습니다.  

    마찬가지로 dataset_test_prt_result() 의 경우에는 테스트 데이터를 바탕으로 진행된 결과를 출력하기에 어떠한 모델을 사용했는지, 
    평가에 소요된 시간은 얼마나 되는지를 함께 출력하도록 설계하였습니다. 

    최종 출력결과를 함께 살펴보면 조금 더 쉽게 이해할 수 있습니다. 
    하단에 학습에 걸린 총 시간 같은 경우 mlp_model_train() 의 모든 반복 학습이 완료되었을 때 출력되어지는 문자열이기에 
    당연히 mlp_model_test() 메서드의 dataset_test_prt_result() 결과는 그 아래에 출력되어 집니다.


``` python
class Model(object):
    …
    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001,  report=0, show_cnt=3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()


def mlp_model_train(self, epoch_count=10, batch_size=10, learning_rate=0.001, report=0):
    …
    for epoch in range(epoch_count):
        …
        for n in range(batch_count):
        …
        if report > 0 and (epoch+1) % report == 0:
            …
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)


MlpModel.train = mlp_model_train  # Model class의 train에서 돌아감

def dataset_train_prt_result(self, epoch, costs, accs, acc, time1, time2):  
    print('	Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)'.format(
        epoch, np.mean(costs), np.mean(accs), acc, time1, time2))

Dataset.train_prt_result = dataset_train_prt_result

```